In [1]:
import os
os.environ['HTTP_PROXY'] = "127.0.0.1:33210"
os.environ['HTTPS_PROXY'] = "127.0.0.1:33210"
os.environ["OPENAI_API_KEY"] = "you-api_key"


from langchain.document_loaders import PyPDFLoader

from PyPDF2 import PdfMerger
file_list = ["KT820说明书V2.0 201909.pdf","GSK980TC3系列  编程操作手册（2018年6月第3版）0622定(1).pdf"]
file_merger = PdfMerger()       # 合并pdf文件
for pdf in file_list:
    file_merger.append(pdf)
# 合并后的pdf放在本地
file_merger.write("merge.pdf")
loader = PyPDFLoader("merge.pdf")
unprocessed_documents = loader.load()


In [44]:
# 用户输入的问题
question = "KT820和GSK980TC3是什么"

import jieba
# 利用jieba_fast进行分词
def jieba_extract_keywords(question):
    # 使用jieba进行分词
    keywords = jieba.cut(question)
    return keywords

# 利用chatgpt进行分词
def gpt_extract_keywords(question):

    from langchain.prompts import ChatPromptTemplate
    prompts = f"""
            ----
            {question}
            ----
            你是一个专家级的文本关键词提取大师，根据上面的question，从其中提取关键词并一行一行地返回它们。关键词提取应该尽可能全面，包括单个有意义的词汇和有意义的词组。

            下面这个是正确示范，仅作为参考，不用返回:
            question: 请告诉我G100代码有哪些功能
            正确输出:
            G100
            代码
            G100代码
            功能

            正确示范:
            question: 报警号197是什么
            正确输出:
            报警号
            197
            报警号197

            错误示范：
            question: 使用手册的版权归谁所有
            错误输出:
            使用手册的版权
            归谁所有
            正确输出：
            使用手册
            版权
            使用手册的版权
            归谁所有

            错误示范:
            question: GSK980TC3的报警号0001是什么
            正确输出：
            GSK980TC3
            报警号
            0001
            报警号0001


    """

    promptsTemplate = ChatPromptTemplate.from_template(prompts)
    from langchain.chains import LLMChain
    from langchain.chat_models import ChatOpenAI
    llm = ChatOpenAI(model_name='gpt-3.5-turbo-16k', temperature=0)
    chain = LLMChain(llm=llm, prompt=promptsTemplate)
    response = chain.run(question=question)
    return response

In [45]:
import PyPDF2
def search_keywords_in_pdf(doc,keywords):
    keyword_count_per_page = {}
    for page_num in range(len(doc)):
        # 计算当前页面的关键词出现次数
        count = sum(doc[page_num].page_content.count(keyword) for keyword in keywords)

        # 如果关键词出现次数大于0，则记录该页码和次数
        if count > 0:
            keyword_count_per_page[page_num+1] = count
    # 根据关键词出现次数从大到小排序页面号
    sorted_page_numbers = [k for k, v in sorted(keyword_count_per_page.items(), key=lambda item: item[1],reverse=True)]
    return sorted_page_numbers

def get_page_content(file_path, page_number):
    with open(file_path, 'rb') as file:
        reader = PyPDF2.PdfReader(file)

        # 确保页面号在有效范围内
        if page_number - 1 < len(reader.pages):
            page = reader.pages[page_number - 1]
            text = page.extract_text()
            return text
        else:
            return "指定的页码超出了PDF文件的范围"


In [46]:
import time

start_time = time.time()
# 从问题中提取关键词
keywords = gpt_extract_keywords(question)
print(keywords)
import ast
keywords = keywords.splitlines()

print(keywords)
print(type(keywords))

KT820
GSK980TC3
['KT820', 'GSK980TC3']
<class 'list'>


In [47]:
os.environ["OPENAI_API_KEY"] = "you-api_key"
from langchain.chat_models import ChatOpenAI


In [48]:
pages = search_keywords_in_pdf(unprocessed_documents,keywords)
print(f"关键词匹配的页面有: {pages}")

关键词匹配的页面有: [249, 408, 411, 178, 409, 251, 268, 10, 180, 248, 271, 427, 506, 250, 253, 255, 257, 259, 261, 262, 263, 265, 266, 267, 269, 270, 273, 275, 277, 279, 281, 283, 285, 287, 289, 291, 293, 295, 297, 299, 301, 303, 305, 307, 309, 311, 313, 315, 317, 319, 321, 323, 325, 327, 329, 331, 333, 335, 337, 339, 341, 343, 345, 347, 349, 351, 353, 355, 357, 359, 361, 363, 365, 367, 369, 371, 373, 375, 377, 379, 381, 383, 385, 387, 389, 391, 393, 395, 397, 399, 401, 403, 405, 407, 413, 415, 416, 417, 419, 421, 423, 425, 429, 430, 431, 433, 435, 437, 439, 441, 443, 445, 447, 449, 451, 453, 455, 457, 459, 461, 463, 465, 467, 469, 471, 473, 475, 477, 479, 481, 483, 485, 487, 489, 491, 493, 494, 495, 497, 499, 501, 503, 505, 507, 508, 509, 510, 511, 512, 513, 514, 515, 516, 517, 518, 519, 520, 521, 522, 523, 524, 525, 526, 527, 528, 529, 530, 531, 532, 533, 534, 535, 536, 537, 538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 548, 549, 550, 551, 552, 553, 555, 557, 559, 561, 563]


In [49]:
documents = []
pages = pages[:50]

from collections import defaultdict
# 使用 defaultdict 存储未处理的文档，键为页面编号，值为文档列表
unprocessed_documents_dict = defaultdict(list)
for doc in unprocessed_documents:
    unprocessed_documents_dict[doc.metadata['page']].append(doc)

# 使用列表推导式来按页面顺序添加文档
documents = [doc for page in pages for doc in unprocessed_documents_dict[page - 1] if page - 1 in unprocessed_documents_dict]

print(documents)
print(len(documents))

[Document(page_content=' \n 245附    \n录 附录： 12 KT820/830Ti 伺服放线图  \nRVVP10*0.2mm\nFG15黑\n双色蓝\n黄色\n红色\n红黑25FG红\n金属壳体\n红蓝色\n伺\n服\n驱\n动KT820Ti\nDB25直针\nCN2华大伺服电机\nPE01221112\n23240105\n10\n21\n09\n0820\n19\n07Z+B-A-\nB+A+0V5V\nZ-\nU+\nU-\nV-V+\nW+\nW-W-15W+12V+11\n14V-U-1310U+Z-09025V\n0V03\n04A+\nB+0507A-\n08B-\n06Z+华大伺服电机\n4\n132\nPEWVUSG-30ADB25直孔\nCN1\n04+24V 08COM+DG03CZ02 PC03\n14OV 05CZCOMALM15 DALM05SIGN-07 CW-10\n07EN 21SONPLUS-06 CP-0902CW+ 19SIGN+PLUS+18 CP+01CN11/CN13DB15直针\nKT-820Ti\nSFFX-KT820Ti\nSFFX-KT820Ti\n3F\n0EDC\n12BA\n型号:\n图号：更改文件号\n4设 计\n审 核标记\n主管设计\n工 艺\n5处数KT820\n830Ti车床配伺服放线\n共  1  页图 样 标 记日期\n日 期\n6签 字\n修改日期\n批 准审 定名称:\n第     页\n7重 量\n1\n8比 例\n9FEDCBA\n粉\n粉黑\n棕\n棕白橙黑橙\n蓝\n蓝白\n白\n白黑\n紫\n紫白\n屏蔽紫色\n黑色橙色\n白色灰色绿色\n屏蔽RVV4*1.5mm护套线\nRVVP7*2*0.2MM\n金属壳体屏蔽棕绿\nGNDCALMGNDSVC_OUT1\n04CALM\n25GND主\n轴\n模\n拟\n量DB25针CN15\n白\nGND12SVC_OUT1\n黄13RVVP4*0.2mm*3 m/根LIM+37贴\n定\n义\n标\n志\n贴,\n粘\n锡\n03DIQPDIQP\nLIM+DECX04DECX\nDECZ12DECZCN61\nDB44直孔\n输\n入\n口T0431T033029T02T0116\nOV21黄色蓝色

In [50]:
from langchain.text_splitter import CharacterTextSplitter
os.environ["OPENAI_API_KEY"] = "you-api_key"

text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

from langchain.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

from langchain.vectorstores import Chroma

db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever()

from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-0613"), chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

prompt = f"""
    下面是用户提的问题,以'''为分割符号
    '''
    {question}
    '''
    回答要求语气温和有礼貌、并且回答内容要详细、有具体文本支撑观点。
"""

res = qa(prompt)
answer, docs = res['result'], res['source_documents']
end_time = time.time()
execution_time = end_time - start_time
print(f"代码运行时间: {execution_time} 秒")
print(answer)


代码运行时间: 14.93616771697998 秒
KT820和GSK980TC3是数控系统的型号。KT820是一种数控系统，而GSK980TC3是另一种数控系统。这两种数控系统都用于车床机床的控制。

根据提供的文本，GSK980TC3是一种总线式车床数控系统，采用了800×600分辨率的彩色8.4英寸液晶显示器。它的面板分为LCD液晶显示区、编辑键盘区、软键功能区和机床控制区四个区域。

至于KT820，根据提供的文本没有提到具体的功能和特点，所以无法给出更详细的信息。
